In [1]:
import numpy as np
import pandas as pd
import os
import glob

In [2]:
doc_folder = './doc/'
dsn_cahier_technique = 'dsn_cahier_technique'
excel_extension = '.xlsx'

print(f'Available files in {doc_folder}:')
for file_name in [fn for fn in os.listdir(doc_folder) if not fn.startswith('.')]:
    print('   ', file_name)

dsn_excel = glob.glob(doc_folder+dsn_cahier_technique+'*'+excel_extension)
assert len(dsn_excel) != 0, f'No excel file containing "{dsn_cahier_technique}" found in "{doc_folder}" folder'
assert len(dsn_excel) < 2, f'More than one excel file containing "{dsn_cahier_technique}" found in "{doc_folder}" folder'

dsn_excel = dsn_excel[0]
print('\nExcel file found:')
print('   ', dsn_excel, end='\n\n')

dsn_dfs = pd.read_excel(dsn_excel, sheet_name=None)

def print_data_frames(dfs, verbose=False):
    for (df_name, df) in dfs.items():
        if verbose:
            print(df_name + '_'*(100-len(df_name)))
            print(df.head(5), '\n')
        else:
            print(f"{df_name:15}   [{str(df.columns)}]")
            
print_data_frames(dsn_dfs)


Available files in ./doc/:
    dsn_cahier_technique_p3_2019.1.2.pdf
    dsn_cahier_technique_p3_2019.1.2.xlsx

Excel file found:
    ./doc\dsn_cahier_technique_p3_2019.1.2.xlsx

Standard          [Index(['DSN'], dtype='object')]
Header            [Index(['Element', 'Id', 'Name', 'Description', 'Comment'], dtype='object')]
Data Types        [Index(['Id', 'Nature', 'Regexp', 'Lg Min', 'Lg Max', 'Values'], dtype='object')]
Blocks            [Index(['Id', 'Name', 'Description', 'ParentId', 'lowerBound', 'upperBound'], dtype='object')]
Fields            [Index(['Block Id', 'Id', 'Name', 'Description', 'DataType Id', 'Comment'], dtype='object')]
Messages          [Index(['Name', 'Description', 'Message'], dtype='object')]


In [3]:
class Block:
    id_blocks = {}
    full_name_blocks = {}
    
    def __init__(self, id, name, description='', lower_bound=1, upper_bound=1):
        self.id = id
        self.name = name
        self.full_name = name
        self.description = description
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.description = description
        self.children = []
        self.rubriques = []
        Block.id_blocks[id] = self
        Block.full_name_blocks[name] = self
        
    def __str__(self):
        return f'[{self.id}] {self.name:44} ({str(self.lower_bound):s}, {str(self.upper_bound):s})'
    
    def __repr__(self):
        return f'{type(self).__name__}(id={self.id}, name={self.name}, {self.lower_bound}, {self.upper_bound})' 
    
    def append(self, id, name, description='', lower_bound=1, upper_bound=1):
        self.children.append(Block(id, name, description, lower_bound, upper_bound))
        
    @classmethod
    def append_in_parent(cls, id_parent, id, name, description='', lower_bound=1, upper_bound=1):
        cls.id_blocks[id_parent].append(id, name, description, lower_bound, upper_bound)
    
    @classmethod
    def append_rubrique(cls, id_block, id, name, full_name, description, id_data_type):
        i = name.find('.')
        name_rubrique = name[i+1:]
        name_block = name[:i]
        cls.id_blocks[id_block].name = name_block
        cls.id_blocks[id_block].rubriques.append(Rubrique(id_block, id, name_rubrique, full_name, description, id_data_type))
        
    def deep_print(self, depth=0, print_rubriques=False):
        print(' '*8*depth + ('' if not depth else'└─ ') + str(self)) # '├─ '
        [print(' '*8*depth + '   > ' + str(r)) for r in self.rubriques if print_rubriques]
        [b.deep_print(depth+1, print_rubriques) for b in self.children] 
            
class Rubrique:
    id_rubriques = {}
    name_rubriques = {}
    
    def __init__(self, id_block, id, name, full_name, description, id_data_type):
        self.id = id
        self.name = name
        self.full_name =  full_name
        self.description = description
        self.id_data_type = id_data_type
        self.id_block = id_block
        Rubrique.id_rubriques[id_block+str(id)] = self
        Rubrique.name_rubriques[name] = self
    
    def __str__(self):
        return f"{'['+str(self.id)+']':5} {self.name:40}"
    
    def __repr__(self):
        return f'{type(self).__name__}(id={self.id}, name={self.name})' 
    
    def deep_print(self, depth=0, print_rubriques=False, last_children=False):
        print(' '*8*depth + ('' if not depth else('└─ ' if last_children else '└─ ')) + repr(self)) # '├─ '
        
        for b in self.children:
            b.deep_print(depth+1, last_children=b==self.children[-1])
            
    def data_type(self):
        return DataType.id_data_types[self.id_data_type]
            
class DataType:
    id_data_types = {}
    
    def __init__(self, id, nature, regex, lg_min, lg_max, values):
        self.id = id
        self.nature = nature
        self.regex = regex
        self.lg_min, self.lg_max = lg_min, lg_max
        self.values = [tuple(v.split('=')) for v in values.split(';')] if values is not np.nan else []
        DataType.id_data_types[id] = self
        
class RubriqueConf:
    
    def __init__(self, rubrique, is_included_by_default=True, use_default_value=False, default_value=''):
        self.rubrique = rubrique
        self.is_included_by_default = is_included_by_default
        self.use_default_value = use_default_value
        self.default_value = default_value
        
    def __repr__(self):
        return f'{type(self).__name__}(id={self.rubrique.id}, name={self.rubrique.name})'
        
class RubriqueValue:
    
    def __init__(self, rubrique_conf, value='', is_included=None):
        self.rubrique_conf = rubrique_conf
        if is_included is None:
            self.is_included = rubrique_conf.is_included_by_default
        else:
            self.is_included = bool(is_included)
        if value != '':
            self.value = value
        elif rubrique_conf.use_default_value:
            self.value = rubrique_conf.default_value
            
    def __repr__(self):
        return f'{type(self).__name__}(id={self.rubrique_conf.rubrique.id}, name={self.rubrique_conf.rubrique.name})'

class BlockConf:
    
    def __init__(self, block):
        self.block = block
        self.rubriques = [RubriqueConf(r) for r in self.block.rubriques]
        self.sub_blocks = [BlockConf(b)  for b in block.children]
        
    def __repr__(self):
        return f'{type(self).__name__}(id={self.block.id}, name={self.block.name})'
            
class BlockValue:
    
    def __init__(self, block_conf, number=None):
        self.block_conf = block_conf
        self.instances = []
        if number is None:
            self.number = self.block_conf.block.lower_bound
        else:
            self.number = number
        if self.number:
            for i in range(self.number):
                self.instances.append((self.create_instance_rubriques(), 
                                       self.create_instance_sub_blocks()))
            
    def __repr__(self):
        return '{2}(id={1.id}, name={1.name}, number={0.number})'.format(
            self, self.block_conf.block, type(self).__name__)
        
    def create_instance_rubriques(self):
        return [RubriqueValue(r) for r in self.block_conf.rubriques]
    
    def create_instance_sub_blocks(self):
        return [BlockValue(b) for b in self.block_conf.sub_blocks]  

In [4]:
root = Block(id='DSN', name='DSN_Root', )
root.append('S10.G00.00', 'Envoi')
root.append('S20.G00.05', 'Déclaration')
root.append('S90.G00.90', "Total de l'envoi") 

df_blocks = dsn_dfs['Blocks']
df_rubriques = dsn_dfs['Fields']
df_data_type = dsn_dfs['Data Types']

fd_blocks_index_mapping = {'Id':'id', 'Name':'name', 'ParentId':'id_parent', 'lowerBound':'lower_bound', 
                           'upperBound':'upper_bound', 'Description':'description'}
for index, row in df_blocks.iterrows():
    args = {fd_blocks_index_mapping[key]:value for key, value in row.items()}
    Block.append_in_parent(**args)
    
fd_rubriques_index_mapping = {'Block Id':'id_block', 'Id':'id', 'Name':'full_name', 'Description':'description',
                              'DataType Id':'id_data_type', 'Comment':'name'}
for index, row in df_rubriques.iterrows():
    args = {fd_rubriques_index_mapping[key]:value for key, value in row.items()}
    Block.append_rubrique(**args)
    
fd_data_type_index_mapping = {'Id':'id', 'Nature':'nature', 'Regexp':'regex', 
                              'Lg Min':'lg_min', 'Lg Max':'lg_max', 'Values':'values'}
for index, row in df_data_type.iterrows():
    args = {fd_data_type_index_mapping[key]:value for key, value in row.items()}
    DataType(**args)


In [5]:
print('Block name max lenght')
max_lenght_block = max(len(i.name) for i in Block.id_blocks.values())
[print(i, max_lenght_block) for i in Block.id_blocks.values() if len(i.name) == max_lenght_block]
print('\nRubrique name max lenght')
max_lenght_rub = max(len(i.name) for i in Rubrique.id_rubriques.values())
[print(i, max_lenght_rub) for i in Rubrique.id_rubriques.values() if len(i.name) == max_lenght_rub]
print('\n\n')
root.deep_print(print_rubriques=False)

Block name max lenght
[S21.G00.16] ChangementsDestinataireAdhesionPrevoyance    (0, *) 41

Rubrique name max lenght
[63]  FPIndiceBrutOrigineAncienSalarieEmployeurPublic 47
[65]  FPMaintienTraitementOrigineContractuelTitulaire 47
[42]  FPMaintienTraitementOrigineContractuelTitulaire 47



[DSN] DSN_Root                                     (1, 1)
        └─ [S10.G00.00] Envoi                                        (1, 1)
                └─ [S10.G00.01] Emetteur                                     (1, 1)
                └─ [S10.G00.02] Contact                                      (1, 1)
        └─ [S20.G00.05] Declaration                                  (1, 1)
                └─ [S20.G00.07] ContactDeclare                               (0, *)
                └─ [S20.G00.08] IdentifiantOrganismeDestinataireNeant        (0, *)
                └─ [S21.G00.06] Entreprise                                   (1, 1)
                        └─ [S21.G00.11] Etablissement                           

In [6]:
bcr = BlockConf(root)

def printBConf(bc):
    print(bc)
    for b in bc.sub_blocks:
        printBConf(b)
        
bvr = BlockValue(bcr)

def printBValue(bc):
    print(bc)
    for rl, bl in bc.instances:
        for sb in bl:
            printBValue(sb)

printBValue(bvr)

BlockValue(id=DSN, name=DSN_Root, number=1)
BlockValue(id=S10.G00.00, name=Envoi, number=1)
BlockValue(id=S10.G00.01, name=Emetteur, number=1)
BlockValue(id=S10.G00.02, name=Contact, number=1)
BlockValue(id=S20.G00.05, name=Declaration, number=1)
BlockValue(id=S20.G00.07, name=ContactDeclare, number=0)
BlockValue(id=S20.G00.08, name=IdentifiantOrganismeDestinataireNeant, number=0)
BlockValue(id=S21.G00.06, name=Entreprise, number=1)
BlockValue(id=S21.G00.11, name=Etablissement, number=1)
BlockValue(id=S21.G00.15, name=PrevoyanceAdhesion, number=0)
BlockValue(id=S21.G00.82, name=CotisationEtablissement, number=0)
BlockValue(id=S21.G00.20, name=VersementOPS, number=0)
BlockValue(id=S21.G00.22, name=CotisationBordereau, number=0)
BlockValue(id=S21.G00.42, name=AffectationFiscale, number=0)
BlockValue(id=S21.G00.44, name=AssujettissementFiscal, number=0)
BlockValue(id=S21.G00.30, name=Individu, number=0)
BlockValue(id=S21.G00.85, name=TravailLieu, number=0)
BlockValue(id=S89.G00.32, name=B

In [7]:
from PyQt5.QtWidgets import *

class BlockFrame(QWidget):
    
    def __init__(self, layout, block, depth=0):
        super().__init__()
        
        #self.frame.resize(300, 36)
        #self.resize(300+20*depth, 54)
        
        self.frame = QFrame()
        self.frame.setFrameStyle(QFrame.Panel | QFrame.Raised)
        self.frame.setLineWidth(2)
        cb = QCheckBox()
        la = QLabel()
        la.setText(repr(block))
        la.setMaximumWidth (400)
        
        frameLayout = QHBoxLayout()
        #frameLayout.setContentsMargins(0, 0, 0, 0)
        frameLayout.addWidget(cb)
        frameLayout.addWidget(la)
        #frameLayout.addStretch()
        self.frame.setLayout(frameLayout)
        
        self.layout = QHBoxLayout()
        self.layout.setContentsMargins(0, 0, 0, 0)
        self.layout.addSpacing(depth*20)
        self.layout.addWidget(self.frame) 
        self.layout.addStretch()
        self.setLayout(self.layout)
        layout.addWidget(self)
        for sub_block in block.children: 
                BlockFrame(layout, sub_block, depth+1)


In [8]:
import sys
from PyQt5.QtWidgets import *
app=QApplication.instance() # checks if QApplication already exists 
if not app: # create QApplication if it doesnt exist 
     app = QApplication(sys.argv)
mainWindow = QMainWindow()
mainWindow.resize(1200, 800)
sa = QScrollArea()
#sa.setGeometry(20, 20, 1160, 760)
sa.setWidgetResizable(True)
sa.setFrameStyle(QFrame.NoFrame)
mainWindow.setCentralWidget(sa)
#saLayout = QHBoxLayout()
#saLayout.addWidget(sa)
#mainWindow.setLayout(saLayout)

w = QWidget()
#w.resize(2600, 800)
sa.setWidget(w)

wLayout = QVBoxLayout()
#wLayout.setContentsMargins(0, 0, 0, 0)



b = BlockFrame(wLayout, root.children[1], 0)
wLayout.addStretch()
w.setLayout(wLayout)
mainWindow.show()
app.exec_()


0